In [ ]:
%pip install -U ipywidgets

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset, load_dataset

dataset_path = "/mnt/ai-stuff-fast/datasets/wikipedia-20231101.en-science-sci-fi-OpenHermes-2.5-chatML-train"
new_dataset_path = "/mnt/ai-stuff-fast/training-results/mini-mistral-wikipedia-20231101.en-science-sci-fi-OpenHermes-2.5-chatML/dataset"
model_path = "/media/gronkomatic/Embiggen/ai-stuff/training-results/runs/run-20240315-211134/checkpoint-70180"

In [ ]:
# 1. Load dataset
# 2. Shuffle
# 3. Save to disk

dataset = load_dataset(dataset_path, save_infos=True)

In [ ]:
dataset["train"][:10]

In [ ]:
dataset = dataset.shuffle(seed=42)

In [ ]:
dataset["train"][:10]

In [ ]:
dataset.save_to_disk(new_dataset_path)

In [ ]:
openhermes_chatml = load_dataset(new_dataset_path)
openhermes_chatml

In [ ]:
openhermes_chatml['train'][0]['text']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.chat_template = "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [ ]:
len(tokenizer)

In [ ]:
dataset = load_dataset(dataset_path)
dataset

In [ ]:
dataset["train"][0]

In [ ]:
dataset = load_dataset("json", data_files=dataset_path)
# dataset = dataset.shuffle()
dataset

In [ ]:
dataset = dataset["train"].select(range(10))
dataset

In [ ]:
# Define a function to rename the keys
def rename_keys(example):
    new_conversations = []
    for conv in example['conversations']:
        if conv["from"] == "gpt":
            conv["from"] = "assistant"
            
        new_conversations.append({
            "role": conv["from"],
            "content": conv["value"]
        })
    return {
        "conversations": new_conversations
    }


# Apply the function to the dataset
renamed_dataset = dataset.map(rename_keys)
renamed_dataset["train"]["conversations"][0]

In [ ]:
dataset = renamed_dataset.map(lambda x: {"text": tokenizer.apply_chat_template(
    x["conversations"], tokenize=False, add_generation_prompt=False)}, remove_columns=["conversations"])
dataset

In [ ]:
dataset["train"][0]

In [ ]:
dataset.save_to_disk(new_dataset_path)